In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import operator
import time
import datetime
from eunjeon import Mecab

ModuleNotFoundError: No module named 'requests'

In [2]:
"""
한겨레 신문사 페이지에서, 특정 검색어 검색 결과 50개의
URL 주소, 기사 제목, 기사 내용을 출력
"""


def hani_search(keyword):
    # 한겨레 신문사 검색 URL
    url = 'http://search.hani.co.kr/Search?command=query&media=news&sort=d&period=all&'
    l = []
    dt = datetime.datetime.now()
    date = dt.strftime('%Y_%m_%d')

    for page in range(5):
        req_params = {
            'keyword': keyword,
            'pageseq': page
            }
        response = requests.get(url, params=req_params)
        html = response.text.strip()
        soup = BeautifulSoup(html, 'html5lib')
        results = soup.select('ul.search-result-list li dt a')
        for link in results:
            d = {}
            d['Link'] = link.get('href')
            d['Title'] = link.text
            d['Article'] = hani_article(d['Link'])
            l.append(d)
            #news_url = link.get('href') 
            #news_title = link.text 
            #print(news_url, news_title)
            #hani_article(news_url)
        df = pd.DataFrame(l)
        df.to_csv(dt + '[' + keyword + ']' + '.csv', mode='a', header=False)
        return df


def hani_article(url):
    response = requests.get(url)
    html = response.text.strip()
    # print(html[:500])
    soup = BeautifulSoup(html, 'html5lib')
    article = soup.select('div.article-text div.text')[0].text.strip()
    article = clean_text(article)
    return article

def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>▶▽♡◀◇━@\#$%&\\\=\(\'\"ⓒ(\n)(\t)]', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace("🇲\u200b🇮\u200b🇱\u200b🇱\u200b🇮\u200b🇪\u200b", "")
    return cleaned_text

In [32]:
if __name__ == '__main__':
    hani_search('사회적 가치 기업')

In [3]:
hani_search('사회적 가치 기업')

NameError: name 'datetime' is not defined